# Radial Basis Function

Σκοπός της παρούσας άσκησης είναι η πρόβλεψη ως προς το αν ένας/μία πελάτης/ισσα μίας τράπεζας θα κάνει τη χρήση της προθεσμιακής κατάθεσης ή όχι. Αυτή η πρόβλεψη θα εξαχθεί με βάση συγκεκριμένα δημογραφικά χαρακτηριστικά του/της
όπως η ηλικία, η εργασία, η εκπαίδευση, η οικογενειακή κατάσταση κ.λπ.
Για να το επιτύχουμε αυτό θα κάνουμε χρήση μίας εφαρμογής βασισμένη σε RBF. Στο κρυφό επίπεδο των νευρώνων υπολογίζονται τα κέντρα των clusters με τη χρήση του αλγορίθμου k-means.


 

**Βήμα 1ο**: Αρχικά φορτώνουμε τις απαράιτητες βιβλιοθήκες

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import math
from sklearn.metrics import accuracy_score

**Βήμα 2ο**: διάβαζουμε το αρχείο bank-full.csv και απομονώνουμε τις στήλες με τα χαρακτηριστικά που θα χρησιμοποιήσουμε (εργασία, αν έχει δανειακές υποχρεώσεις κλπ). Το αρχείο είναι διαθέσιμο [εδώ](https://raw.githubusercontent.com/nkostopoulos/StochasticsLabPublic/master/lab9/bank-full.csv) και θα πρέπει να το φορτώσετε στο colab.

In [2]:
dataset = pd.read_csv('bank-full.csv',sep=';')

In [3]:
dataset.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [4]:
col_to_use = ['age','balance','day','duration','campaign','pdays','previous']
data = dataset.drop(col_to_use,axis=1)

In [5]:
data.head()

,job,marital,education,default,housing,loan,contact,month,poutcome,y
0,management,married,tertiary,no,yes,no,unknown,may,unknown,no
1,technician,single,secondary,no,yes,no,unknown,may,unknown,no
2,entrepreneur,married,secondary,no,yes,yes,unknown,may,unknown,no
3,blue-collar,married,unknown,no,yes,no,unknown,may,unknown,no
4,unknown,single,unknown,no,no,no,unknown,may,unknown,no


In [6]:
data = data.apply(LabelEncoder().fit_transform)

In [7]:
data.head()

,job,marital,education,default,housing,loan,contact,month,poutcome,y
0,4,1,2,0,1,0,2,8,3,0
1,9,2,1,0,1,0,2,8,3,0
2,2,1,1,0,1,1,2,8,3,0
3,1,1,3,0,1,0,2,8,3,0
4,11,2,3,0,0,0,2,8,3,0


In [8]:
data['job'].unique()

array([ 4,  9,  2,  1, 11,  5,  0,  7,  6, 10,  3,  8])

In [9]:
dataset['job'].unique()

array(['management', 'technician', 'entrepreneur', 'blue-collar',
       'unknown', 'retired', 'admin.', 'services', 'self-employed',
       'unemployed', 'housemaid', 'student'], dtype=object)

In [10]:
data_rest = dataset[col_to_use]
data_rest.head()

,age,balance,day,duration,campaign,pdays,previous
0,58,2143,5,261,1,-1,0
1,44,29,5,151,1,-1,0
2,33,2,5,76,1,-1,0
3,47,1506,5,92,1,-1,0
4,33,1,5,198,1,-1,0


In [11]:
dataset2 = pd.concat([data_rest,data],axis=1)

In [12]:
dataset2.head()

,age,balance,day,duration,campaign,pdays,previous,job,marital,education,default,housing,loan,contact,month,poutcome,y
0,58,2143,5,261,1,-1,0,4,1,2,0,1,0,2,8,3,0
1,44,29,5,151,1,-1,0,9,2,1,0,1,0,2,8,3,0
2,33,2,5,76,1,-1,0,2,1,1,0,1,1,2,8,3,0
3,47,1506,5,92,1,-1,0,1,1,3,0,1,0,2,8,3,0
4,33,1,5,198,1,-1,0,11,2,3,0,0,0,2,8,3,0


**Βήμα 3ο**: Σπάμε στο dataset σε train και test dataset και κάνουμε scale με τη χρήση της συνάρτησης StandardScaler()

In [13]:
X= dataset2.drop('y',axis=1)
y= dataset2['y']

X_train, X_test, y_train, y_test= train_test_split(X,y, test_size= 0.33, random_state= 4)

In [14]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [15]:
X_train

array([[ 3.39736083, -0.45852779,  0.14087303, ..., -0.71332324,
        -1.50513854,  0.44672656],
       [-0.27605947,  2.43941798,  0.50067953, ...,  0.40010942,
        -1.83733874, -2.58685457],
       [ 0.66584317, -0.13326522, -0.93854645, ..., -0.71332324,
         1.48466323, -0.56446715],
       ...,
       [-0.74701079, -0.22619739,  0.62061503, ..., -0.71332324,
         1.15246303,  0.44672656],
       [ 1.60774581, -0.42275894, -1.53822394, ...,  1.51354209,
         0.15586244,  0.44672656],
       [ 0.28908211, -0.13627382,  1.58009901, ..., -0.71332324,
        -0.17633776,  0.44672656]])

**Βήμα 4ο**: Ορίζουμε τα κέντρα των νευρώνων με τη χρήση του k-means. Αρχικά τρέχει ο αλγόριθμος k-means για να βρούμε τα κέντρα και την τυπική απόκλιση των clusters.

In [16]:
K_cent = 8
km = KMeans(n_clusters=K_cent, max_iter=100)
km.fit(X_train)
cent = km.cluster_centers_

In [17]:
# Η παράμετρος max άλλαξε σε max_param, διότι η μεταβλητή max είναι εγγενής της Python
max_param=0 
for i in range(K_cent):
    for j in range(K_cent):
        d = np.linalg.norm(cent[i]-cent[j])
        if(d > max_param):
            max_param = d
d = max_param

sigma= d/math.sqrt(2*K_cent)

**Βήμα 5ο**: Δημιουργούμε τον πίνακα σχέσεων F (γραμμικός συνδυασμός των Κ συναρτήσεων βάσης φj(x).

In [18]:
shape = X_train.shape
row = shape[0]
column = K_cent
F = np.empty((row,column), dtype= float)

In [19]:
for i in range(row):
    for j in range(column):
        dist= np.linalg.norm(X_train[i]-cent[j])
        F[i][j]= math.exp(-math.pow(dist,2)/math.pow(2*sigma,2))

**Βήμα 6ο**: Βρίσκουμε τον πίνακα βάρους W για την εκπαίδευση του δικτύου

In [20]:
FTG = np.dot(F.T,F)
FTG_inv = np.linalg.inv(FTG)
fac = np.dot(FTG_inv,F.T)
W = np.dot(fac,y_train)

**Βήμα 7ο**: Ρύθμιση του πίνακα F για το test dataset

In [21]:
row= X_test.shape[0]
column= K_cent
F_test= np.empty((row,column), dtype= float)
for i in range(row):
    for j in range(column):
        dist= np.linalg.norm(X_test[i]-cent[j])
        F_test[i][j]= math.exp(-math.pow(dist,2)/math.pow(2*sigma,2))

**Βήμα 8ο**: Ακρίβεια πρόβλεψης στο test dataset

In [22]:
prediction= np.dot(F_test,W)
prediction= 0.5*(np.sign(prediction-0.5)+1)

score= accuracy_score(prediction,y_test)
print(score.mean())

0.8876675603217158


## Ερωτήσεις

**Ερώτηση 1**: Να αναφέρετε επιγραμματικά με ποιον τρόπο λειτουργεί ένα νευρωνικό δίκτυο RBF

**Ερώτηση 2**:Με ποιούς τρόπους μπορούμε να υπολογίσουμε τα βάρη στα hidden layers; Ποιες διαφορές υπάρχουν στους τρόπους αυτούς; Ποιος παρέχει τη καλύτερη δυνατή λύση;

**Ερώτηση 3**: Επιχειρήστε να αλλάξετε την τιμή του k σε 10 και στη συνέχεια σε 50. Τι παρατηρείτε ως προς την ακρίβεια των αποτελεσμάτων;

### Απαντήσεις

- <font color='#486393'>Ένα Νευρωνικό Δίκτυο RBF (RBFN) αποτελεί ειδική περίπτωση νευρωνικού δικτύου στο οποίο ως ενεργοποιήσεις χρησιμοποιούνται ακτινικές συναρτήσεις βάσης (radial basis functions), δηλαδή βαθμωτές συναρτήσεις $\phi\left(\mathbf{x}\right)$, με $\mathbf{x} \in \mathbb{R}^d$, οι οποίες έχουν εξάρτηση μόνο από την απόσταση του ορίσματός τους από κάποιο σταθερό σημείο $\mathbf{c}$. Με άλλα λόγια, ισχύει</font>

<font color='#486393'>$$ \phi\left(\mathbf{x}\right) = \phi\left(||\mathbf{x}-\mathbf{c} ||\right), $$</font>

<font color='#486393'>με το $\mathbf{c}$ να καλείται το κέντρο (center) της συνάρτησης. Τα RBFN αποτελούνται τυπικά από 3 επίπεδα: το επίπεδο εισόδου (input layer), ένα κρυφό επίπεδο (hidden layer) το οποίο περιλαμβάνει τις μη-γραμμικές ενεργοποιήσεις ακτινικών συναρτήσεων βάσης και ένα γραμμικό επίπεδο εξόδου (output layter). Έτσι, εάν η είσοδος αντιστοιχεί στο διάνυσμα $\mathbf{x}^\text{in}$, τότε η έξοδος του RBFN θα είναι η βαθμωτή ποσότητα</font>

<font color='#486393'>$$ x^\text{out} = w_0 + \sum_{i=1}^{N}{w_{i}\phi_i\left(||\mathbf{x}^\text{in}-\mathbf{c}_i||\right)}, $$</font>

<font color='#486393'>η οποία (π.χ. μέσω του προσήμου της) επιτρέπει την ταξινόμηση του δειγματικού σημείου εισόδου σε μια από δύο πιθανές κατηγορίες (υπάρχουν και γενικεύσεις σε διανυσματικές εξόδους, όπου η ταξινόμηση πραγματοποιείται σε μία κατηγορία ανάμεσα σε περισσότερες από δύο). Στη σχέση αυτή, η άθροιση γίνεται επάνω σε όλα τα συναπτικά βάρη $w_i$ μεταξύ εισόδου και κρυφού επιπέδου (τα οποία θεωρούμε πως είναι $N$ σε πλήθος) επί τις αντίστοιχες ακτινικές συναρτήσεις βάσης. Η παράμετρος $w_0$ αντιστοιχεί στο bias που προστίθεται στην έξοδο. Σημειώνεται πως μια από τις συνηθέστερες επιλογές ακτινικής συνάρτησης βάσης είναι η Γκαουσιανή, γνωστή και ως Γκαουσιανός πυρήνας, δηλαδή η</font>

<font color='#486393'>$$ \phi\left(\mathbf{x}\right) \propto \exp\left(-\frac{||\mathbf{x}-\mathbf{c} ||^2}{2\sigma^2}\right), $$</font>

<font color='#486393'>με $\sigma$ την τυπική απόκλιση και $\mathbf{c}$ τη μέση τιμή (η οποία ταυτίζεται με το κέντρο της συνάρτησης).</font>

- <font color='#486393'>Σε ό,τι έχει να κάνει με τον υπολογισμό των βαρών $w_i$ του κρυφού επιπέδου, δηλαδή την ίδια την εκπαίδευση του δικτύου, μπορεί κανείς να διακρίνει ανάμεσα σε δύο ευρύτερες οικογένειες μεθόδων στα πλαίσια του υπό μελέτη προβλήματος.</font>

<font color='#486393'>Σε ό,τι έχει να κάνει με την πρώτη μέθοδο, το δίκτυο μπορεί να εκπαιδευτεί μέσω των γνωστών αλγορίθμων (π.χ. Gradient Descent) με τυχαίες αρχικοποιήσεις των ακτινικών συναρτήσεων βάσης. Η δεύτερη προσέγγιση είναι γνωστή και ως υβριδική εκμάθηση (hybrid learning) και είναι μάλιστα αυτή η οποία ακολουθείται και στο παρόν notebook. Η υβριδική εκμάθηση αποτελείται από ένα στάδιο μη επιβλεπόμενης εκμάθησης και ένα στάδιο επιβλεπόμενης εκμάθησης. Το πρώτο στάδιο συνιστά τον προσδιορισμό των ακτινικών συναρτήσεων βάσης του κρυφού επιπέδου αξιοποιώντας τα δεδομένα εκπαίδευσης. Στα πλαίσια της παρούσας άσκησης, η αρχικοποίηση αυτή αφορούσε στον προσδιορισμό των κέντρων των Γκαουσιανών πυρήνων και κατόπιν τις αντίστοιχες τιμές για την τυπική απόκλιση, χρησιμοποιώντας τον αλγόριθμο συσταδοποίησης k-Means. Κατά το δεύτερο στάδιο, με δεδομένες τις υπολογισμένες τιμές για τα κέντρα και τις αποκλίσεις από αυτά, το δίκτυο εκπαιδεύεται με τις γνωστές μεθόδους επιβλεπόμενης μάθησης.</font>

<font color='#486393'>Όπως γίνεται αντιληπτό, η βασική διαφορά ανάμεσα στις δύο μεθόδους έγκειται στην αρχικοποίηση των ακτινικών συναρτήσεων βάσης, η οποία γίνεται με τυχαίο τρόπο στην πρώτη και μέσω χρήσης των δεδομένων εκπαίδευσης στη δεύτερη. Προφανώς, η δεύτερη μέθοδος είναι προτιμότερη, αφού μια κακή αρχικοποίηση των παραμέτρων των ακτινικών συναρτήσεων βάσης μπορεί να επηρεάσει αρνητικά την εκπαίδευση του δικτύου (για παράδειγμα παγίδευση σε τοπικά ελάχιστα κατά τη διαδικασία Gradient Descent). Μάλιστα, κατά τη μέθοδο υβριδικής εκμάθησης υπάρχει η δυνατότητα οι υπολογισμένες παράμετροι να αξιοποιηθούν απλώς ως αρχικές τιμές οι οποίες μπορούν να υποστούν επιπλέον ρύθμιση (fine-tuning) κατά τη διάρκεια της επιβλεπόμενης μάθησης.</font>

- <font color='#486393'>Προκειμένου να πραγματοποιηθούν τα δύο πειράματα για τις δοσμένες τιμές του $k$, ορίζουμε πρώτα μία ενιαία συνάρτηση (για λόγους συντομίας) που να πραγματοποιεί τα παραπάνω βήματα σειριακά.</font>

In [23]:
def RBFN(k,X_train,y_train,X_test,y_test):
    K_cent = k
    km = KMeans(n_clusters=K_cent, max_iter=100)
    km.fit(X_train)
    cent = km.cluster_centers_

    max_param = 0
    for i in range(K_cent):
        for j in range(K_cent):
            d = np.linalg.norm(cent[i]-cent[j])
            if(d > max_param):
                max_param = d
    d = max_param

    sigma= d/math.sqrt(2*K_cent)

    shape = X_train.shape
    row = shape[0]
    column = K_cent
    F = np.empty((row,column), dtype=float)

    for i in range(row):
        for j in range(column):
            dist = np.linalg.norm(X_train[i]-cent[j])
            F[i][j] = math.exp(-math.pow(dist,2)/math.pow(2*sigma,2))

    FTG = np.dot(F.T,F)
    FTG_inv = np.linalg.inv(FTG)
    fac = np.dot(FTG_inv,F.T)
    W = np.dot(fac,y_train)

    row = X_test.shape[0]
    column = K_cent
    F_test = np.empty((row,column), dtype=float)
    for i in range(row):
        for j in range(column):
            dist = np.linalg.norm(X_test[i]-cent[j])
            F_test[i][j] = math.exp(-math.pow(dist,2)/math.pow(2*sigma,2))

    prediction = np.dot(F_test,W)
    prediction = 0.5*(np.sign(prediction-0.5)+1)

    score = accuracy_score(prediction,y_test)
    return score.mean()

<font color='#486393'>Δεδομένης της συνάρτησης αυτής, αφότου επαναληφθεί ο υπολογισμός για $k = 8$ ως ένα sanity check, η διαδικασία πραγματοποιείται για $k = 10$ και κατόπιν για $k = 50$. Πέραν των τελικών αποτελεσμάτων για την ακρίβεια της ταξινόμησης καταγράφεται και ο συνολικός χρόνος εκτέλεσης.</font>

In [24]:
%%time
k_val = 8
Score_8 = RBFN(k_val,X_train,y_train,X_test,y_test)
print(f'Η ακρίβεια στα δεδομένα αξιολόγησης για k = {k_val} είναι ίση με {Score_8*100:.3f}%.')

Η ακρίβεια στα δεδομένα αξιολόγησης για k = 8 είναι ίση με 88.767%.
Wall time: 3.07 s


In [25]:
%%time
k_val = 10
Score_10 = RBFN(k_val,X_train,y_train,X_test,y_test)
print(f'Η ακρίβεια στα δεδομένα αξιολόγησης για k = {k_val} είναι ίση με {Score_10*100:.3f}%.')

Η ακρίβεια στα δεδομένα αξιολόγησης για k = 10 είναι ίση με 89.042%.
Wall time: 3.7 s


In [26]:
%%time
k_val = 50
Score_50 = RBFN(k_val,X_train,y_train,X_test,y_test)
print(f'Η ακρίβεια στα δεδομένα αξιολόγησης για k = {k_val} είναι ίση με {Score_50*100:.3f}%.')

Η ακρίβεια στα δεδομένα αξιολόγησης για k = 50 είναι ίση με 89.551%.
Wall time: 19.9 s


<font color='#486393'>Στον ακόλουθο πίνακα καταγράφονται τα αποτελέσματα για την ακρίβεια ανά τιμή του $k$, καθώς και για το συνολικό χρόνο εκτέλεσης του αλγορίθμου.</font>

| Τιμή $k$ | Τελική ακρίβεια | Χρόνος εκτέλεσης |
| :---: | :---: | :--- |
| 8 | 88.767% | 3.07 s |
| 10 | 89.042% | 3.7 s |
| 50 | 89.551% | 19.9 s |

<font color='#486393'>Το πρώτο σημαντικό συμπέρασμα είναι πως η αύξηση του $k$ οδηγεί σε αισθητή αύξηση του χρόνου εκτέλεσης του αλγορίθμου, όχι τόσο κατά την αύξηση $k = 8 \to k = 10$, οι οποίες είναι συγκρίσιμες τιμές, όσο κατά την αύξηση $k = 10 \to k = 50$. Πέρα από το συνολικό χρόνο εκτέλεσης, όμως, παρατηρείται και μια αύξηση στην τελική ακρίβεια του μοντέλου, όπως αυτή υπολογίζεται βάσει των προβλέψεών του στα δεδομένα του συνόλου αξιολόγησης. Αν και η αύξηση αυτή δεν είναι πολύ υψηλή (η αύξηση είναι της τάξης του 0.5% σε κάθε περίπτωση), αυτό μάλλον οφείλεται μερικώς και στο γεγονός πως οι τιμές της ακρίβειας είναι ήδη πολύ υψηλές. Στο επόμενο κελί ο αλγόριθμος εκτελείται για ακόμη μία φορά, με $k = 500$, προκειμένου να φανεί πως ο κορεσμός στην τελική τιμή της ακρίβειας δεν έχει επέλθει ακόμα (δηλαδή, μπορεί να αυξηθεί κι άλλο), όμως το υπολογιστικό κόστος αρχίζει μάλλον να υπερβαίνει την ανάγκη για υψηλότερη ακρίβεια, αφού αυτά τα δύο δεν είναι ανάλογα.</font>

In [27]:
%%time
k_val = 500
Score_500 = RBFN(k_val,X_train,y_train,X_test,y_test)
print(f'Η ακρίβεια στα δεδομένα αξιολόγησης για k = {k_val} είναι ίση με {Score_500*100:.3f}%.')

Η ακρίβεια στα δεδομένα αξιολόγησης για k = 500 είναι ίση με 90.275%.
Wall time: 2min 59s


<font color='#486393'>Πράγματι, παρότι η τελική ακρίβεια αυξήθηκε κατά περίπου 0.7% σε σχέση με την τιμή που βρέθηκε στην περίπτωση $k = 50$, ο υπολογιστικός χρόνος επίσης αυξήθηκε σημαντικά, φτάνοντας περίπου τα 3 λεπτά. Έτσι, η χρήση $k = 50$ ή και ενδεχομένως $k = 10$ να είναι προτιμότερη, αφού δεν πρέπει κανείς να ξεχνά πως η υψηλή αύξηση του αριθμού των συστάδων ενδέχεται να επηρεάσει τη γενικευσιμότητα του εκπαιδευμένου μοντέλου λόγω φαινομένων υπερπροσαρμογής.</font>